<a href="https://colab.research.google.com/github/ravi-gopalan/DAND_Data_Wrangling/blob/master/spacy_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [46]:
cd '/gdrive/My Drive/abv_reviews'

/gdrive/My Drive/abv_reviews


In [3]:
ls

images/  pending_images/  reviews_text.csv


In [0]:
import pandas as pd
import numpy as np
import random
import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
import spacy
import spacy.cli
spacy.cli.download("en_core_web_lg")
import en_core_web_lg
from spacy import displacy
from spacy.matcher import Matcher
# Import the English language class
from spacy.lang.en import English

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [49]:
!pip install spacymoji
from spacymoji import Emoji

In [0]:
nlp = spacy.load('en_core_web_lg', disable = ['ner'])
emoji = Emoji(nlp)
nlp.add_pipe(emoji, first=True)
matcher = Matcher(nlp.vocab)




In [7]:
df_review = pd.read_csv('reviews_text.csv')
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 36 columns):
Unnamed: 0           56156 non-null int64
_id                  56156 non-null object
updatedAt            56156 non-null object
createdAt            56156 non-null object
originality          56156 non-null int64
value                56156 non-null int64
nutrition            56156 non-null int64
presentation         56156 non-null int64
taste                56156 non-null int64
text                 56155 non-null object
dish                 56156 non-null object
type                 56156 non-null object
isPublished          5885 non-null object
user                 56156 non-null object
restaurant           56156 non-null object
images               56156 non-null object
likes                56156 non-null object
__v                  56156 non-null int64
comments             56156 non-null object
likesCount           56156 non-null int64
tags                 56156 non-null obje

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,25,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
review_text = df_review[['_id','text']]
review_text.head()

,_id,text
0,59e599911747c90004ba8586,Burger joint offers a wide range of cheeseburg...
1,59ff122b8b04fd0004df6c78,It was really good. The mushroom broth was esp...
2,5dc251e552e7e90020b6adfb,"$8.90 for sesame rice, mushroom rendang, curry..."
3,5d35cf4049745d0004c68de3,2 mains + 1 green rice bento from greendot. Re...
4,5d304e9264f5e5000423b75b,The lion mane mushroom rendang is so delicious...


In [0]:
# Combine the # and tag to form hashtag

matcher.add('HASHTAG', None, [{'ORTH': '#'}, {'IS_ASCII': True}])

doc = nlp('This is a #sentence. Here is another #hashtag. #The #End.')
matches = matcher(doc)
hashtags = []
for match_id, start, end in matches:
    hashtags.append(doc[start:end])

for span in hashtags:
    span.merge()

print([t.text for t in doc])

In [68]:
# Select a random review from the corpus
check = random.randint(0,20000)
review_sample = review_text['text'][check:check+5]
#print(review_sample)

# convert that into nlp
df_token = pd.DataFrame()

stop_word_list = []
punctuation_list = []
non_english_list = []
det_list = []
adj_list = []
noun_list = []
adverb_list = []
verb_list = []
other_pos_list = []
prop_noun_list = []
hashtags = []

for sample in review_sample:
  review_doc = nlp(sample)

# Break those into sentences
  print('----')
  for sentence in review_doc.sents:
    print(sentence.text)

  matches = matcher(review_doc)

  for match_id, start, end in matches:
    hashtags.append(review_doc[start:end])

  for span in hashtags:
    span.merge()


# Find the part of speech
  print('----')

  tokens_list = []
  for token in review_doc:
    token_dict = {}
    token_dict['text'] = token.text
    token_dict['lemma'] = token.lemma_
    token_dict['part_of_speech'] = token.pos_
    token_dict['dependency'] = token.dep_
    token_dict['shape'] = token.shape_
    token_dict['alphabet?'] = token.is_alpha
    token_dict['stop_word'] = token.is_stop
    token_dict['punctuation'] = token.is_punct
    token_dict['digit'] = token.is_digit
    token_dict['language'] = token.lang_
    token_dict['prefix'] = token.prefix_
    token_dict['sentiment'] = token.sentiment
    token_dict['currency'] = token.is_currency

    tokens_list.append(token_dict)
 
    if token.is_stop:
      stop_word_list.append(token)
    if token.is_punct:
      punctuation_list.append(token)
    if token.lang_ != 'en':
      non_english_list.append(token)
    if token.pos_ == 'DET':
      det_list.append(token)
    elif token.pos_ == 'ADJ':
      adj_list.append(token.lemma_)
    elif token.pos_ == 'NOUN':
      noun_list.append(token.lemma_)
    elif token.pos_ == 'ADV':
      adverb_list.append(token.lemma_)
    elif token.pos_ == 'VERB':
      verb_list.append(token.lemma_)
    elif token.pos_ == 'PROPN':
      prop_noun_list.append(token.lemma_)
    else:
      other_pos_list.append(token)

    df_token = pd.concat([df_token, pd.DataFrame(tokens_list)],axis=0)

print("Noun List: {}".format(set(noun_list)))
print("Proper Noun List: {}".format(set(prop_noun_list)))
print("Adjective List: {}".format(set(adj_list)))
print("Verb List: {}".format(set(verb_list)))
print("Adverb List: {}".format(set(adverb_list)))
print("Det List: {}".format(set(det_list)))
print("Other POS List: {}".format(set(other_pos_list)))
print("Non-English: {}".format(set(non_english_list)))
print("Punctuations: {}".format(set(punctuation_list)))
print("Stop Words: {}".format(set(stop_word_list)))

df_token

----
This was heavenly.
The cookies made the waffles so fantastic and so much sauce too.
----
----
$6.50! mmm
my favorite waffles
SO GOOD
the outside was crispy and the inside was fluffy and soft!!
the mango added a nice touch to it as well:D also the description said gula melaka
but honestly it tasted like maple syrup though,
so it was honestly like the maple drizzle waffles (which were $4) and some mango:/ but still really good nonetheless!!
will probably get the maple drizzle one for more value
----
----
🇸🇬
Se pasaron estos waffles
❤
❤
❤
Los ame, crujientes pero suaves
y con el dulzor perfecto!!!
----
----
This satisfied my sweet tooth 🦷
And it didn’t taste of coconuts which is a win.
Will be back to try the rest of their vegan desserts!!
----
----
Had the awesome waffles again!
Hot, fluffy and crispy but slightly soggy because of the maple syrup that it was drenched in.
Maybe next time I'll tell them to give me the maple syrup separately so it won't be so soggy.
----
Noun List: {'d

,text,lemma,part_of_speech,dependency,shape,alphabet?,stop_word,punctuation,digit,language,prefix,sentiment,currency
0,This,this,DET,nsubj,Xxxx,True,True,False,False,en,T,0.0,False
0,This,this,DET,nsubj,Xxxx,True,True,False,False,en,T,0.0,False
1,was,be,VERB,ROOT,xxx,True,True,False,False,en,w,0.0,False
0,This,this,DET,nsubj,Xxxx,True,True,False,False,en,T,0.0,False
1,was,be,VERB,ROOT,xxx,True,True,False,False,en,w,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,n't,not,ADV,neg,x'x,False,True,False,False,en,n,0.0,False
43,be,be,VERB,advcl,xx,True,True,False,False,en,b,0.0,False
44,so,so,ADV,advmod,xx,True,True,False,False,en,s,0.0,False
45,soggy,soggy,ADJ,acomp,xxxx,True,False,False,False,en,s,0.0,False


In [69]:
hashtags



[]

In [0]:
similarity_list = []
for token1 in review_doc:
  for token2 in review_doc:
    sim_dict = {}
    sim_dict['word_1'] = token1.text
    sim_dict['word_2'] = token2.text

    try:
      sim_dict['similarity'] = token1.similarity(token2)
    except:
      print('error')


    similarity_list.append(sim_dict)

df_similarity = pd.DataFrame(similarity_list)\
.query('similarity < 0.9999')\
.sort_values(['similarity'],ascending=False)
df_similarity

/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty v

,word_1,word_2,similarity
194,broc,cauli,0.649460
222,cauli,broc,0.649460
198,and,for,0.570359
58,for,and,0.570359
5,love,!,0.450742
...,...,...,...
177,x,broc,-0.147135
213,cauli,for,-0.182374
59,for,cauli,-0.182374
183,broc,for,-0.194480


In [41]:
displacy.render(nlp("Seitan hot dog was really good and flavourful with a good bounce that you want in a hot dog."),'dep',True,0,jupyter=True)

In [0]:
matcher = Matcher(nlp.vocab)
# Write a pattern for ice cream
pattern_ice_cream = [{"TEXT": "ice"}, {"TEXT": "cream"}]
pattern_laksa = [{"TEXT": "laksa"}]
pattern_pizza = [{"TEXT": "pizza"}]

# Add the pattern to the matcher and apply the matcher to the doc
#matcher.add('EMOJI', None, [{'IS_EMOJI':True}])
matcher.add('HASHTAG', None, [{'ORTH': '#'}, {'IS_ASCII': True}])
matcher.add('hyphenated',None,[{'IS_ASCII': True},{'ORTH': '-'},{'IS_ASCII': True}])
matcher.add("propn_adj_noun", None, [{'POS': 'PROPN'}, {'POS': 'ADJ'}, {'POS': 'NOUN'}])
matcher.add("adj_noun", None, [{'POS': 'NOUN'}, {'POS': 'NOUN'}])
matcher.add("portobello mushroom",None,[{'LOWER': 'portobello'},{'LOWER': 'mushroom'}])
matcher.add("any_chicken", None, [{"POS": "NOUN"}, {"LOWER": "chicken"}])
matcher.add("any_salad", None, [{"POS": "NOUN"}, {"LOWER": "salad"}])
matcher.add("any_burger", None, [{"POS": "NOUN"}, {"LOWER": "burger"}])
matcher.add("soy_sauce", None, [{"LOWER": "soy"}, {"LOWER": "sauce"}])
matcher.add("green_curry", None, [{"LOWER": "green"}, {"LOWER": "curry"}])
matcher.add("any_soup", None, [{"POS": "NOUN"}, {"LOWER": "soup"}])
matcher.add("any_cream", None, [{"POS": "NOUN"}, {"LOWER": "cream"}])
matcher.add("sticky_rice", None, [{"LOWER": "sticky"}, {"LOWER": "rice"}])
matcher.add("hor_fun", None, [{"LOWER": "hor"}, {"LOWER": "fun"}])
#matcher.add("laksa", None, pattern_laksa)
#matcher.add("pizza", None, pattern_pizza)

In [33]:
sentence1 = "Seitan hot dog was really good and flavourful with a good bounce that you want in a hot dog."
sentence2 = "This burger is one of the reasons why non-vegans say vegan food tastes bad."

sent_list = [sentence1, sentence2]

for sentence in sent_list:
  matches = matcher(nlp(sentence))
  print("Total matches found:", len(matches))

# Iterate over the matches and print the span text
  for match_id, start, end in matches:
    print("Match found:", nlp(sentence)[start:end].text)

Total matches found: 1
Match found: Seitan hot dog
Total matches found: 2
Match found: non-
Match found: non-vegans


In [0]:
doc = nlp("A phrase with another phrase occurs.")
len(doc.noun_chunks_)


AttributeError: ignored

In [0]:
list(nlp(sentence1).noun_chunks)
list(nlp(sentence2).noun_chunks)

[Seitan hot dog, a good bounce, you, a hot dog]

[Vegan risotto]

In [0]:
matcher.add("any_curry", None, [{"POS": "NOUN"}, {"LOWER": "curry"}])
matcher.add("any_rice", None, [{"POS": "NOUN"}, {"LOWER": "rice"}])
matcher.add("hor_fun", None, [{"LOWER": "hor"}, {"LOWER": "fun"}])
matcher.add("any_chutney", None, [{"POS": "NOUN"}, {"LOWER": "chutney"}])
matcher.add("any_pudding", None, [{"POS": "NOUN"}, {"LOWER": "pudding"}])
matcher.add("grilled_any", None, [{"LOWER": "grilled"}, {"POS": "NOUN"}])
matcher.add("fried_any", None, [{"LOWER": "fried"}, {"POS": "NOUN"}])
matcher.add("roasted_any", None, [{"LOWER": "roasted"}, {"POS": "NOUN"}])
matcher.add("baked_any", None, [{"LOWER": "baked"}, {"POS": "NOUN"}])
matcher.add("frosted_any", None, [{"LOWER": "frosted"}, {"POS": "NOUN"}])
matcher.add("mashed_any", None, [{"LOWER": "mashed"}, {"POS": "NOUN"}])
matcher.add("sautéed_any", None, [{"LOWER": "sautéed"}, {"POS": "NOUN"}])
matcher.add("any_wings", None, [{"POS": "NOUN"}, {"LOWER": "wings"}])
matcher.add("any_chorizo", None, [{"POS": "NOUN"}, {"LOWER": "chorizo"}])
matcher.add("any_cheese", None, [{"POS": "NOUN"}, {"LOWER": "cheese"}])
matcher.add("any_croissant", None, [{"POS": "NOUN"}, {"LOWER": "croissant"}])
matcher.add("any_nuggets", None, [{"POS": "NOUN"}, {"LOWER": "nuggets"}])


In [71]:
review_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56156 entries, 0 to 56155
Data columns (total 2 columns):
_id     56156 non-null object
text    56155 non-null object
dtypes: object(2)
memory usage: 877.6+ KB


In [0]:
doc = nlp(u"This is a test 😻 👍🏿")
assert doc._.has_emoji == True
assert doc[2:5]._.has_emoji == True
assert doc[0]._.is_emoji == False
assert doc[4]._.is_emoji == True
assert doc[5]._.emoji_desc == u'thumbs up dark skin tone'

In [0]:
matcher.add("any_sprouts", None, [{"POS": "NOUN"}, {"LOWER": "sprouts"}])
matcher.add("any_burrito", None, [{"POS": "NOUN"}, {"LOWER": "burrito"}])
matcher.add("any_juice", None, [{"LOWER": "hor"}, {"LOWER": "juice"}])
matcher.add("any_flakes", None, [{"POS": "NOUN"}, {"LOWER": "flakes"}])
matcher.add("any_pasta", None, [{"POS": "NOUN"}, {"LOWER": "pasta"}])
matcher.add("any_popiah", None, [{"POS": "NOUN"}, {"LOWER": "popiah"}])
matcher.add("any_tzatziki", None, [{"POS": "NOUN"}, {"LOWER": "tzatziki"}])
matcher.add("any_tomatoes", None, [{"POS": "NOUN"}, {"LOWER": "tomatoes"}])
matcher.add("any_rendang", None, [{"POS": "NOUN"}, {"LOWER": "rendang"}])
matcher.add("any_sorbet", None, [{"POS": "NOUN"}, {"LOWER": "sorbet"}])
matcher.add("any_pancake", None, [{"POS": "NOUN"}, {"LOWER": "pancake"}])
matcher.add("any_quiche", None, [{"POS": "NOUN"}, {"LOWER": "quiche"}])
matcher.add("any_yogurt", None, [{"POS": "NOUN"}, {"LOWER": "yogurt"}])
matcher.add("any_dumpling", None, [{"POS": "NOUN"}, {"LOWER": "dumpling"}])
matcher.add("any_salsa", None, [{"POS": "NOUN"}, {"LOWER": "salsa"}])
matcher.add("any_butter", None, [{"POS": "NOUN"}, {"LOWER": "butter"}])
matcher.add("any_rings", None, [{"POS": "NOUN"}, {"LOWER": "rings"}])
matcher.add("any_fries", None, [{"POS": "NOUN"}, {"LOWER": "fries"}])
matcher.add("any_bento", None, [{"POS": "NOUN"}, {"LOWER": "bento"}])
matcher.add("any_lassi", None, [{"POS": "NOUN"}, {"LOWER": "lassi"}])
matcher.add("any_flower", None, [{"POS": "NOUN"}, {"LOWER": "flower"}])
matcher.add("any_patties", None, [{"POS": "NOUN"}, {"LOWER": "patties"}])
matcher.add("any_roll", None, [{"POS": "NOUN"}, {"LOWER": "roll"}])
matcher.add("any_sandwich", None, [{"POS": "NOUN"}, {"LOWER": "sandwich"}])
matcher.add("any_gelato", None, [{"POS": "NOUN"}, {"LOWER": "gelato"}])
matcher.add("any_browns", None, [{"POS": "NOUN"}, {"LOWER": "browns"}])
matcher.add("any_cake", None, [{"POS": "NOUN"}, {"LOWER": "cake"}])
matcher.add("any_pastry", None, [{"POS": "NOUN"}, {"LOWER": "pastry"}])
matcher.add("any_biscuit", None, [{"POS": "NOUN"}, {"LOWER": "biscuit"}])
matcher.add("any_mian", None, [{"POS": "NOUN"}, {"LOWER": "mian"}])
matcher.add("any_dressing", None, [{"POS": "NOUN"}, {"LOWER": "dressing"}])
matcher.add("any_bun", None, [{"POS": "NOUN"}, {"LOWER": "bun"}])
matcher.add("any_syrup", None, [{"POS": "NOUN"}, {"LOWER": "syrup"}])
matcher.add("any_bowl", None, [{"POS": "NOUN"}, {"LOWER": "bowl"}])
matcher.add("any_wedges", None, [{"POS": "NOUN"}, {"LOWER": "wedges"}])
matcher.add("any_balls", None, [{"POS": "NOUN"}, {"LOWER": "balls"}])
matcher.add("any_bacon", None, [{"POS": "NOUN"}, {"LOWER": "bacon"}])
matcher.add("any_choi", None, [{"POS": "NOUN"}, {"LOWER": "choi"}])
matcher.add("any_chips", None, [{"POS": "NOUN"}, {"LOWER": "chips"}])
matcher.add("any_ravioli", None, [{"POS": "NOUN"}, {"LOWER": "ravioli"}])
matcher.add("any_dosai", None, [{"POS": "NOUN"}, {"LOWER": "dosai"}])
matcher.add("any_vindaloo", None, [{"POS": "NOUN"}, {"LOWER": "vindaloo"}])
matcher.add("any_doughnuts", None, [{"POS": "NOUN"}, {"LOWER": "doughnuts"}])
matcher.add("any_donuts", None, [{"POS": "NOUN"}, {"LOWER": "donuts"}])
matcher.add("any_gnocchi", None, [{"POS": "NOUN"}, {"LOWER": "gnocchi"}])
matcher.add("any_vinaigrette", None, [{"POS": "NOUN"}, {"LOWER": "vinaigrette"}])

matcher.add("vegetarian", None, [{"LOWER": "vegetarian"}])
matcher.add("vegan", None, [{"LOWER": "vegan"}])
matcher.add("meatless", None, [{"LOWER": "meatless"}])
matcher.add("omnivore", None, [{"LOWER": "omnivore"}])

matcher.add("any_sticks", None, [{"POS": "NOUN"}, {"LOWER": "sticks"}])
matcher.add("any_shell", None, [{"POS": "NOUN"}, {"LOWER": "shell"}])
matcher.add("any_wrap", None, [{"POS": "NOUN"}, {"LOWER": "wrap"}])
matcher.add("any_sausage", None, [{"POS": "NOUN"}, {"LOWER": "sausage"}])
matcher.add("any_scramble", None, [{"POS": "NOUN"}, {"LOWER": "scramble"}])
matcher.add("any_smoothie", None, [{"POS": "NOUN"}, {"LOWER": "smoothie"}])

matcher.add("any_cookie", None, [{"POS": "NOUN"}, {"LOWER": "cookie"}])
matcher.add("any_coffee", None, [{"POS": "NOUN"}, {"LOWER": "coffee"}])
matcher.add("any_latte", None, [{"POS": "NOUN"}, {"LOWER": "latte"}])
matcher.add("any_mocha", None, [{"POS": "NOUN"}, {"LOWER": "mocha"}])
matcher.add("any_free", None, [{"POS": "NOUN"}, {"LOWER": "free"}])
matcher.add("any_chai", None, [{"POS": "NOUN"}, {"LOWER": "chai"}])
matcher.add("any_tofu", None, [{"POS": "NOUN"}, {"LOWER": "tofu"}])


matcher.add("street_food", None, [{"LOWER": "street"}, {"LOWER": "food"}])
matcher.add("restaurant", None, [{"LOWER": "restaurant"}])

In [195]:
review_text.describe(include='all')

,_id,text
count,56156,56155
unique,56156,56072
top,5d603555c7b502000473ebfb,nice.............................................
freq,1,9


In [196]:
# Select a random review from the corpus
check = random.randint(0,50000)
review_sample = review_text['text'][0:10000]
#print(review_sample)



matches_list = []
if len(review_sample) != 0:

  for sample in review_sample:
    review_doc = nlp(sample)
    matches = matcher(review_doc)
  #  print(len(matches), sample )

  # Iterate over the matches and print the span text
    for match_id, start, end in matches:
      matches_list.append(review_doc[start:end].text)


set(matches_list)

TypeError: ignored

In [0]:
from IPython.display import HTML as html_print

def style(s, bold=False):
  blob = f"<text>{s}</text>"
  if bold:
    blob = f"<b style='background-color: #fff59d' > {blob}</b>"
  return blob

def html_generator(g, n):
  blob = ""
  for i in range(n):
    doc = next(g)

    state = [[t, False] for t in doc]
    for idx, start, end in matcher(doc):
      for i in range(start, end):
        state[i][1] = True
        blob += style(' '.join([style(str(t[0]), bold = t[1]) for t in state]) + '<br>')
  return blob

In [95]:
texts = (_ for _ in review_text['text'])
g = (d for d in nlp.pipe(texts) if len(matcher(d)) > 0)

html_print(html_generator(g, n=1))
